# R包randomForest的随机森林回归模型以及对重要变量的选择

> https://mp.weixin.qq.com/s/BWIyVWfCyK09S34p-sIkXg

## 示例数据
植物根系菌群结构与植物生长密切相关，到目前为止，已经在许多研究中都有报道了，这已经是个共识。因此，下文的示例数据也同样来自某植物根际区域细菌群落组成的16S扩增子测序数据，类似地，仿照上文文献中的过程，通过随机森林建立微生物与植物生长时期的响应关系，根据微生物丰度推断植物生长时期，并寻找一组重要的时间特征类群。

示例数据“otu_table.txt”中，共记录了45个连续生长时间中植物根际土壤样本中细菌OTU的相对丰度信息。
“plant_age.txt”中，记录了这45个根际土壤样本对应的植物生长时间，时间单位是天。

## 随机森林回归模型的初步构建



In [ ]:
##数据预处理
#读取 OTUs 丰度表
otu <- read.delim('otu_table.txt', row.names = 1)
 
#过滤低丰度 OTUs 类群，它们对分类贡献度低，且影响计算效率
#例如剔除总丰度低于 0.05% 的值
otu <- otu[which(rowSums(otu) >= 0.0005), ]
 
#合并有关于植物生长时间的信息
plant <- read.delim('plant_age.txt', row.names = 1)
 
otu <- data.frame(t(otu))
otu <- otu[rownames(plant), ]
otu <- cbind(otu, plant)
 
#为了方便后续评估随机森林模型的性能
#将总数据集分为训练集（占 70%）和测试集（占 30%）
set.seed(123)
train <- sample(nrow(otu), nrow(otu)*0.7)
otu_train <- otu[train, ]
otu_test <- otu[-train, ]
 
##randomForest 包的随机森林
library(randomForest)
 
#随机森林计算（默认生成 500 棵决策树），详情 ?randomForest
set.seed(123)
otu_train.forest <- randomForest(plant_age~., data = otu_train, importance = TRUE)
> otu_train.forest

Call:
 randomForest(formula = plant_age ~ ., data = otu_train, importance = TRUE)
               Type of random forest: regression
                     Number of trees: 500
No. of variables tried at each split: 869

          Mean of squared residuals: 165.2854
                    % Var explained: 89.17

In [ ]:

结果中，% Var explained体现了预测变量（用于回归的所有OTU）对响应变量（植物年龄）有关方差的整体解释率。在本示例中，剔除了低丰度的OTU后，剩余的OTU（约2600个左右）解释了约89.17%的总方差，可以理解为该回归的R2=0.8917，相当可观的一个数值，表明该植物根际细菌的群落结构随植物生长密切相关。

查看该模型的预测性能，可以看到具有较高的精度。



In [ ]:
#使用训练集，查看预测精度
plant_predict <- predict(otu_train.forest, otu_train)
 
plot(otu_train$plant_age, plant_predict, main = '训练集',
    xlab = 'Plant age (days)', ylab = 'Predict')
abline(1, 1)
 
#使用测试集，评估预测性能
plant_predict <- predict(otu_train.forest, otu_test)
 
plot(otu_test$plant_age, plant_predict, main = '测试集',
    xlab = 'Plant age (days)', ylab = 'Predict')
abline(1, 1)

## 重要的预测变量选择


但是，并非这所有的2600余个OTU都对回归的精度具有可观的贡献。有些OTU丰度的时间特征并不明显，可能在回归中产生较大噪声，对模型精度带来较高的误差。因此，最好将低贡献的OTU去除。

评估预测变量的重要性

基于已经构建好的随机森林回归模型，可以从中评估OTU的重要性。将OTU按重要程度高低进行排名后，选择排名靠前的一部分OTU，这些重要的OTU是明显的与植物生长时间密切关联的一些细菌类群。

In [ ]:
##OTU 的重要性评估
#查看表示每个预测变量（细菌 OTU）重要性的得分
#summary(otu_train.forest)
#importance_otu <- otu_train.forest$importance
importance_otu <- data.frame(importance(otu_train.forest), check.names = FALSE)
 > head(importance_otu)
          %IncMSE IncNodePurity
OTU_2 224.6598863   3683.612114
OTU_3  -1.2679399     29.158000
OTU_4  -0.0320000      0.384000
OTU_5   0.4081606      7.001871
OTU_7  60.1625965   1516.594168
OTU_8   2.7316408     71.912158
#作图展示 top30 重要的 OTUs
varImpPlot(otu_train.forest, n.var = min(30, nrow(otu_train.forest$importance)),
    main = 'Top 30 - variable importance')

“%IncMSE”即increase in mean squared error，通过对每一个预测变量随机赋值，如果该预测变量更为重要，那么其值被随机替换后模型预测的误差会增大。因此，该值越大表示该变量的重要性越大；

“IncNodePurity”即increase in node purity，通过残差平方和来度量，代表了每个变量对分类树每个节点上观测值的异质性的影响，从而比较变量的重要性。该值越大表示该变量的重要性越大。

对于“%IncMSE”或“IncNodePurity”，二选一作为判断预测变量重要性的指标。需注意的是，二者的排名存在一定的差异。



In [ ]:
#可以根据某种重要性的高低排个序，例如根据“IncNodePurity”指标
importance_otu <- importance_otu[order(importance_otu$IncNodePurity, decreasing = TRUE), ]
head(importance_otu)

## 交叉验证

那么，最终选择多少重要的预测变量（本示例为OTUs）是更合适的呢？

可通过执行十折交叉验证，根据交叉验证曲线对OTU进行取舍。交叉验证法的作用就是尝试利用不同的训练集/验证集划分来对模型做多组不同的训练/验证，来应对单独测试结果过于片面以及训练数据不足的问题。此处使用训练集本身进行交叉验证。

In [ ]:
set.seed(123)
otu_train.cv <- replicate(5, rfcv(otu_train[-ncol(otu_train)], otu_train$plant_age, cv.fold = 10, step = 1.5), simplify = FALSE)
otu_train.cv

#提取验证结果绘图
otu_train.cv <- data.frame(sapply(otu_train.cv, '[[', 'error.cv'))
otu_train.cv$otus <- rownames(otu_train.cv)
otu_train.cv <- reshape2::melt(otu_train.cv, id = 'otus')
otu_train.cv$otus <- as.numeric(as.character(otu_train.cv$otus))
 
otu_train.cv.mean <- aggregate(otu_train.cv$value, by = list(otu_train.cv$otus), FUN = mean)
head(otu_train.cv.mean, 10)
 
#拟合线图
library(ggplot2)
 
ggplot(otu_train.cv.mean, aes(Group.1, x)) +
geom_line() +
theme(panel.grid = element_blank(), panel.background = element_rect(color = 'black', fill = 'transparent')) +
labs(title = '',x = 'Number of OTUs', y = 'Cross-validation error')


交叉验证曲线展示了模型误差与用于拟合的OTU数量之间的关系。误差首先会随OTUs数量的增加而减少，开始时下降非常明显，但到了特定范围处，下降幅度将不再有显著变化，甚至有所增加。

根据交叉验证曲线，提示保留9-13个重要的OTU即可以获得理想的回归结果，因为此时的误差达到最小。

因此，根据计算得到的各OUT重要性的值（如“IncNodePurity”），将OTU由高往低排序后，最后大约选择前9-13个OTU就可以了。

In [ ]:
#提示保留 9-13 个重要的 OTU，可以使随机森林回归的精度最大化
#首先根据某种重要性的高低排个序，例如根据“IncNodePurity”指标
importance_otu <- importance_otu[order(importance_otu$IncNodePurity, decreasing = TRUE), ]
 
#然后取出排名靠前的 OTU，例如 top10 最重要的 OTU
importance_otu.select <- importance_otu[1:10, ]
importance_otu.select
 
#输出表格
#write.table(importance_otu.select, 'importance_otu.select.txt', sep = '\t', col.names = NA, quote = FALSE)
 
#有关 OTU 的物种分类信息等其它细节，可后续自行补充